In [1]:
# Install necessary libraries
!pip install transformers sentence-transformers faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 19.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manyli

In [2]:
# Import necessary libraries
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
# Load the pre-trained model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
# Sample texts
texts = [
    "This is an English text.",
    "Este es un texto en español.",
    "Ceci est un texte en français.",
    "Dies ist ein deutscher Text.",
    "これは日本語のテキストです。",
    "这是中文文本。"
]

In [5]:
# Encode texts into vectors
vectors = model.encode(texts)

In [6]:
# Create a FAISS index
dimension = vectors.shape[1]  # dimension of the vectors
index = faiss.IndexFlatL2(dimension)

In [7]:
# Add vectors to the index
index.add(vectors)

In [8]:
# Store vectors with their corresponding texts
vector_text_mapping = {i: text for i, text in enumerate(texts)}

In [9]:
# Query the vector database
def query_database(query, top_k=3):
    query_vector = model.encode([query])
    distances, indices = index.search(query_vector, top_k)
    results = [(vector_text_mapping[idx], distances[0][i]) for i, idx in enumerate(indices[0])]
    return results


In [10]:
# Example query
query = "This is a text in English."
results = query_database(query)




### Interpretation

- **High Similarity**: The result "This is an English text." is very similar to the query "This is a text in English." This is reflected by the low distance score, indicating the model successfully identified a semantically similar text in the database.
  
- **Low Similarity**: The other results "这是中文文本。" (Chinese text) and "これは日本語のテキストです。" (Japanese text) have much higher distance values, indicating they are not semantically similar to the query text.



In [11]:
# Print the results
print(f"Query: {query}\n")
for result, distance in results:
    print(f"Result: {result}, Distance: {distance}")


Query: This is a text in English.

Result: This is an English text., Distance: 4.239224433898926
Result: 这是中文文本。, Distance: 42.41073989868164
Result: これは日本語のテキストです。, Distance: 46.11396026611328
